In [1]:
##Importando os pacotes e módulos necessários
import pandas as pd
import numpy as np
import math

#Para Regressão Linear Simples e Teste F
from scipy import stats
#Para Regressão Linear Múltipla
import statsmodels.api as sm
import econtools
import econtools.metrics as mt

#Pacotes para gráficos
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

#Pacotes para fazer a coleta dos dados armazenados no mesmo diretório e outros pacotes gerais
import os
import pathlib
import glob
from IPython.display import clear_output

##Criando as Funções
def coletar_dados(nome = ""):
    '''
    Função que le os arquivos do Stata
    O arquivo deve estar na mesma pasta do arquivo de Python
    Deixe em branco para ler o arquivo mais recentemente adicionado
    '''
    global df
    #Pegando qual a pasta do arquivo que está sendo usado pra programar
    caminho = pathlib.Path().absolute()

    #checando se o nome foi inserido ou não; caso não, pegar o arquivo .dta mais recente
    if nome == "":
        arquivo = max(glob.glob(f"{str(caminho)}/*.dta"), key=os.path.getctime)
        df = pd.read_stata(arquivo)
        print(f"{arquivo}.dta foi lido com sucesso!")
        return df
    else:
        try:
            arquivo = f"{str(caminho)}/{str(nome)}.dta"
            df = pd.read_stata(arquivo)
            print(f"{nome}.dta foi lido com sucesso!")
            return df
        except: #caso não tenha sido encontrado o arquivo com o nome inserido
            print('''
            Não foi possível achar o arquivo :(\n
            Verifique se seu nome está correto (sem a extensão) e se ele está no mesmo diretório do programa!
            ''')

def Regressão_Simples(Lista_x, Lista_y):
    '''
    Função que calcula as estatísticas de uma regressão simples
    '''
    global Lista_ychapeu_simples, Resíduos_simples
    
    #calculando o número de observações e as médias
    Número_de_Observações = len(Lista_x)
    Média_x = sum(Lista_x)/Número_de_Observações
    Média_y = sum(Lista_y)/Número_de_Observações

    #Calculando os coeficientes do modelo
    B1, B0, R, valor_P, DP = stats.linregress(Lista_x, Lista_y)
    #Gerando os valores previstos
    Lista_ychapeu_simples = [round(B0 + B1 * i,3) for i in Lista_x]
    #Calculando os Resíduos
    Resíduos_simples = [(j - k) for j,k in zip(Lista_y, Lista_ychapeu_simples)]
    
    #Calculando R-quadrados e a Soma dos Quadrados das Variáveis
    R_quadrado = round(R**2,5)
    SQTx = sum([(i - Média_x)**2 for i in Lista_x])
    SQTy = sum([(i - Média_y)**2 for i in Lista_y])
    SQEy = sum([(i - Média_y)**2 for i in Lista_ychapeu_simples])
    SQR = sum([i**2 for i in Resíduos_simples]) 

    #Calculando a Variância da Regressão e dos  Coeficientes
    VarianciaReg = SQR/(Número_de_Observações - 2)
    EPR = math.sqrt(VarianciaReg)
    VarB1 = VarianciaReg/SQTx
    VarB0 = (VarianciaReg * sum([i**2 for i in Lista_x]))/(Número_de_Observações * SQTx)
    
    #Calculando da estatistica t com intervalo de confiança de 95%  (p/ gerar os intervalos de confiança dos estimadores)
    Estatistica_t_Critica = stats.t.ppf(0.95, Número_de_Observações - 2)

    #Calculando os estimadores do limite inferior e superior
    B1_inferior = B1 - math.sqrt(VarB1) * Estatistica_t_Critica
    B1_superior = B1 + math.sqrt(VarB1) * Estatistica_t_Critica
    B0_inferior = B0 - math.sqrt(VarB0) * Estatistica_t_Critica
    B0_superior = B0 + math.sqrt(VarB0) * Estatistica_t_Critica
    
    #Gerando o Relatório
    Relatório = f'''
    Número de Observações = {Número_de_Observações}\n
    B0 = {round(B0,5)}\t B1 = {round(B1,5)}\t R-quadrado = {R_quadrado}\n
    Estimador da Variância = {round(VarianciaReg,5)}\t Erro Padrão da Regressão = {round(EPR,5)}\n
    Variância de B1 = {round(VarB1,5)}\t Variância de B0 = {round(VarB0,5)}\n
    Intervalo de Confiança de 95% para B1 (Inferior; B1; Superior): {round(B1_inferior,4)}; {round(B1,4)}; {round(B1_superior,4)}\n
    Intervalo de Confiança de 95% para B0 (Inferior; B0; Superior): {round(B0_inferior,4)}; {round(B0,4)}; {round(B0_superior,4)}\n
    Para ver os valores previstos, basta chamar a variável 'Lista_ychapeu_simples'\n
    Para ver os resíduos, chame a variável 'Resíduos_simples'
    '''
    print (Relatório)
    
    ##Criando um gráfico
    #Deixando o estilo bonitinho
    sns.set_style(style="white")

    #Criando o objeto gráfico
    Grafico = sns.regplot(x = Lista_x, y = Lista_y, scatter_kws={"color": "black"}, line_kws={"color": "red"})
    Grafico.set_title("Resultado da Regressão",fontsize = 11)
    plt.show()

def Regressão_Múltipla(x,y,constante="S"):
    global Resultado, Lista_ychapeu, Resíduos, SQR, EPR

    #adicionando uma constante ao modelo de Ordinary Least Squares(OLS)
    if constante == "S":
        X = sm.add_constant(x)
    else:
        X = x
    #Criando o Modelo
    Modelo = sm.OLS(y,X)
    Resultado = Modelo.fit()
    
    Lista_ychapeu = Resultado.predict()
    Resíduos = y - Lista_ychapeu

    #Calculando o Erro Padrão da Regressão (EPR)
    SQR =sum([i**2 for i in Resíduos])
    Número_de_Observações = len(y)
    GL = Número_de_Observações - len(Resultado.params)
    VarianciaReg = SQR/GL
    EPR = math.sqrt(SQR/GL)
    
    ##Printando o Resultado
    #print('Parâmetros:\n', Resultado.params) #O primeiro resultado equivale ao intercepto
    #print('\nDesvios Padrões:\n', Resultado.bse)
    #print('Valores Previstos: ', Resultado.predict())
    #print('\nR2:', Resultado.rsquared)
    print(f"O erro padrão da regressão é {round(EPR,5)} e a SQR é {round(SQR,5)}\n")
    print(Resultado.summary())

    print("\nPara ver os valores previstos ou os resídudos, basta chamar 'Lista_ychapeu' e 'Resíduos'.")
    print("Os resultados do modelo podem ser obtidos através de métodos usando a variável 'Resultado'.")
  
def Teste_F(x, y, Restrições, Nivel_de_Significância = 0.05):
    ##Definindo as variáveis de cada modelo
    #para testar igualdade dos coeficientes, F2, p_valueF2 = results.Ftest(['ACT', 'skipped'], equal=True)
    ModeloIrrestrito = list(x)
    ModeloRestrito = []
    Restrições = list(Restrições)

    Numero_de_Observações = len(y)
    GL_ir = Numero_de_Observações - (len(ModeloIrrestrito) + 1)
    GL_r = len(Restrições)

    for i in ModeloIrrestrito:
        if i not in Restrições:
            ModeloRestrito.append(i)

    ##Fazendo as regressões de cada modelo
    Regressão_Múltipla(x, y)
    SQR_ir = SQR
    VarianciaReg_ir = EPR**2

    Regressão_Múltipla(df[ModeloRestrito], y)
    SQR_r = SQR

    #Limpando a tela
    clear_output()
    
    ##Calculando F
    F = (SQR_r - SQR_ir)/(len(Restrições)*VarianciaReg_ir)

    ##Calculando o P-valor de F
    P_valor = stats.f.sf(F,GL_r,GL_ir)

    if Nivel_de_Significância > P_valor:
        print(f"O valor de F é {round(F,3)} e seu p-valor é {round(P_valor,7)}. Portanto, rejeita-se Ho a um nível de significância de {Nivel_de_Significância*100}%.")
    else:
        print(f"O valor de F é {round(F,3)} e seu p-valor é {round(P_valor,7)}. Portanto, não se rejeita Ho a um nível de significância de {Nivel_de_Significância*100}%.")


In [2]:
coletar_dados("GPA3")

GPA3.dta foi lido com sucesso!


,term,sat,tothrs,cumgpa,season,frstsem,crsgpa,verbmath,trmgpa,hssize,...,female,black,white,ctrmgpa,ctothrs,ccrsgpa,ccrspop,cseason,hsperc,football
0,1,920,31,2.25,0,0,2.6464,0.48387,1.50,10,...,1,0,0,NaN,NaN,NaN,NaN,NaN,40.000000,0
1,2,920,43,2.04,1,0,2.5077,0.48387,2.25,10,...,1,0,0,0.75,12.0,-0.1387,-62.250000,1.0,40.000000,0
2,1,780,28,2.03,0,0,2.8679,0.81395,2.20,123,...,0,0,1,NaN,NaN,NaN,NaN,NaN,82.926826,0
3,2,780,43,2.09,1,0,2.8839,0.81395,1.60,123,...,0,0,1,-0.60,15.0,0.0160,-73.250000,1.0,82.926826,0
4,1,810,0,0.00,0,1,2.7634,0.88372,1.60,119,...,0,0,1,NaN,NaN,NaN,NaN,NaN,35.294117,0
5,2,810,14,1.78,1,0,2.5600,0.88372,1.29,119,...,0,0,1,-0.31,14.0,-0.2034,72.329956,1.0,35.294117,0
6,1,1080,0,0.00,1,1,2.5149,0.80000,2.00,318,...,0,0,1,NaN,NaN,NaN,NaN,NaN,9.748427,0
7,2,1080,17,2.00,0,0,2.4764,0.80000,2.73,318,...,0,0,1,0.73,17.0,-0.0385,-320.199951,-1.0,9.748427,0
8,1,960,91,2.35,1,0,2.8535,1.00000,2.80,383,...,0,0,1,NaN,NaN,NaN,NaN,NaN,17.232376,0
9,2,960,106,2.41,0,0,2.7414,1.00000,2.60,383,...,0,0,1,-0.20,15.0,-0.1121,-92.800018,-1.0,17.232376,0


In [9]:
#adicionando as colunas e retirando os dados desncessários
df["female_sat"]= df['female']*df['sat']
df['female_hsperc'] = df['female']*df['hsperc']
df['female_tothrs'] = df['female']*df['tothrs']
df.drop(df[df.term<2].index, inplace = True)

In [40]:
Regressão_Múltipla(df[['female','sat','female_sat','hsperc','female_hsperc','tothrs','female_tothrs']],df['cumgpa'])

O erro padrão da regressão é 0.46783 e a SQR é 78.35451

                            OLS Regression Results                            
Dep. Variable:                 cumgpa   R-squared:                       0.406
Model:                            OLS   Adj. R-squared:                  0.394
Method:                 Least Squares   F-statistic:                     34.95
Date:                Tue, 03 Nov 2020   Prob (F-statistic):           4.66e-37
Time:                        15:04:14   Log-Likelihood:                -237.26
No. Observations:                 366   AIC:                             490.5
Df Residuals:                     358   BIC:                             521.7
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------

In [11]:
#Realizando um teste F convencional
Teste_F(df[['female','sat','female_sat','hsperc','female_hsperc','tothrs','female_tothrs']],df['cumgpa'],df[['female','female_sat','female_hsperc','female_tothrs']])

O valor de F é 8.179 e seu p-valor é 2.5e-06. Portanto, rejeita-se Ho a um nível de significância de 5.0%.


## Estatística de Chow

In [13]:
#Separando os grupos
df_female = df.loc[df["female"] == 1]
df_male = df.loc[df["female"] == 0]

In [16]:
#Estimando cada regressão
Regressão_Múltipla(df_female[['sat','hsperc','tothrs']],df_female['cumgpa'])
SQR1 = SQR
Regressão_Múltipla(df_male[['sat','hsperc','tothrs']],df_male['cumgpa'])
SQR2 = SQR
Regressão_Múltipla(df[['sat','hsperc','tothrs']],df['cumgpa'])
SQRr = SQR

clear_output()

In [17]:
#Calculando a estatística de Chow
F_Chow = (SQRr - (SQR1 + SQR2))/(SQR1 + SQR2)*(358/4)
print(round(F_Chow,4))

8.1791


### Permitindo uma mudança no intercepto

In [18]:
#Realizando o Teste F da significância conjunta dos termos de interação
Teste_F(df[['female','sat','female_sat','hsperc','female_hsperc','tothrs','female_tothrs']],df['cumgpa'],df[['female_sat','female_hsperc','female_tothrs']])

O valor de F é 1.534 e seu p-valor é 0.2053734. Portanto, não se rejeita Ho a um nível de significância de 5.0%.


Não podemos rejeitar Ho usando apenas os termos de interação, mas rejeitamos quando usamos o intercepto diferente.

Assim, o melhor modelo é aquele apenas com interceptos diferentes entre os grupos
